# Entrega 2, Grupo 02 - Aprendizaje Bayesiano

- Santiago Alaniz,  5082647-6, santiago.alaniz@fing.edu.uy
- Bruno De Simone,  4914555-0, bruno.de.simone@fing.edu.uy
- María Usuca,      4891124-3, maria.usuca@fing.edu.uy

## Objetivo

Implementar un algoritmo de predicción de palabras utilizando Naive Bayes. El modelo se entrenará con datos de conversaciones de WhatsApp y se probará en un simulador de cliente. 

El algoritmo debe ser capaz de recomendar palabras basadas en las últimas N palabras ingresadas en una frase, donde N es un hiperparámetro que se variará para evaluar el desempeño del modelo. Además, el modelo se reentrenará al finalizar cada frase para adaptarse a nueva evidencia. 

La implementación debe ser eficiente en términos de uso de CPU, utilizando las estructuras de datos más adecuadas en Python.


## Diseño

### Carga del dump de conversaciones en un grupo de WhatsApp

En la letra del laboratorio se menciona que se puede acceder al dump en formato `.csv` pero esto es un error en la letra. *WhatsApp* exporta las conversaciones en formato `.txt`.

Esto complejiza la carga de los datos ya que no podemos asumir que cada linea del archivo es una entrada del log.

Detectamos el siguiente patron en el archivo de texto

```
  [dd/mm/aaaa hh:mm:ss] <nombre>: <mensaje>
    ***
  [dd/mm/aaaa hh:mm:ss] <nombre>: <mensaje>
```

Nos valemos del modulo `src.regex` para definir la expresion regular que detecta el patron y extrae los mensajes (`LOG_ENTRY_PATTERN`).

`LOG_ENTRY_PATTERN = f'{metadata_pattern}{message_pattern}(?=\n{metadata_pattern}|$)':`

Esta expresion regular captura el contenido de cada entrada del log en dos grupos: `metadata` y `message`. 

- El grupo `metadata` captura la fecha y hora de la entrada, el nombre del usuario o el numero de telefono. 
- El grupo `message` captura el mensaje en si mismo y es el que nos interesa recuperar.

In [2]:
import re
from src.whatsapp_regex import LOG_ENTRY_PATTERN

FILE_PATH = './assets/chat.txt'
PATTERN = LOG_ENTRY_PATTERN

with open(FILE_PATH, 'r', encoding='utf-8') as file:
  data = file.read()

matches = re.findall(PATTERN, data)
data = [ match[1] for match in matches ]

data[:10]

['Horarios disponibles para hoy jueves\nC1️⃣16-17-19-23\nC2️⃣16-17-18-23\nC3️⃣16-17-20-21-23',
 '18 5ta 6ta\n- Diego R\n-\n-\n-',
 '18 5ta 6ta\n- Diego R\n- Facundo B\n-\n-',
 'Hoy 19hs C1 | Cat Princ (primeros partidos)\n\n-Petro\n-Amigo Petro\n-Amigo Petro \n-',
 '18 5ta 6ta\n- Diego R\n- Facundo B\n- Petro\n- Bole',
 'Fabi, vi que reservaste por la app 19h, está perfecto.',
 'Gracias Fer! Ahí vi que la aceptaste 💪🏽',
 'Es importante que cuando vemos los horarios disponibles y nos interesa una hora y de hecho hacemos lista, notificarlo, pedir que la marquemos para asegurarla y/o hacer como Fabi en este caso, usar la app que está perfecto. Entiendo que gente que no tiene la app y hace reservas solo por WA, está perfecto también es nuestro trabajo pero repito, avisen, pidan que la marquemos y no lo dejen al azar porque por ejemplo puede entrar otra persona por la app, reservarla y el partido que estamos armando hace 4 horas se desmorona.',
 '☝🏻👍🏻',
 '18 5ta 6ta\n- Diego R\n- Facundo B\

### Preprocesamiento de datos

Para un algoritmo de sugerencia de palabras basado en aprendizaje bayesiano, el preprocesamiento de los datos es crucial para obtener resultados significativos. En nuestro caso, se compone de los siguientes pasos:

- **Tokenización ordenada**: Para que el algoritmo pueda identificar correctamente las palabras y su posición en la oración, es necesario dividir la oración en [tokens](https://es.wikipedia.org/wiki/Token).
- **Conversión a minúsculas**: Para que el algoritmo no trate las palabras como diferentes solo debido a las diferencias de mayúsculas y minúsculas.
- **Eliminación de caracteres especiales, numericos, emojis**: Estos componentes lexicos y semanticos no aportan información relevante para el modelo que busca sugerir palabras.
- **Relajar tildes**: Para que el modelo no distinga palabras con tildes de palabras sin tildes. Teniendo en cuenta que en los mensajes de WhatsApp es muy común que se omitan las tildes.
- **Eliminación de [stopwords](https://es.wikipedia.org/wiki/Palabra_vac%C3%ADa)**: Estas palabras aparecen con mucha frecuencia en el lenguaje natural independientemente del contexto, por lo que no aportan información relevante para el modelo.
- **Cruzamiento de palabras con un [corpus](http://universal.elra.info/product_info.php?cPath=42_43&products_id=1509)**: Para descartar palabras que no pertenecen al idioma español, chequeamos su existencia en un corpus. Esto es importante para evitar que el modelo sugiera palabras en otros idiomas o palabras que no existen.

Construimos la clase `src.G02Preprocessor` para encapsular el preprocesamiento de los datos y realizar los pasos mencionados anteriormente con mayor facilidad y control.

*Nota:* 

La lista resultado del preprocesamiento tiene un tamaño menor a la lista original. Esto se debe a que el preprocesamiento descarta mensajes que no cumplen con los criterios de preprocesamiento.

In [3]:
from src.preprocessing import G02Preprocessor
import random

preprocessor = G02Preprocessor()
ejemplo = random.choice(data)


print(f'\
    Largo del diccionario: {len(preprocessor.spanish_words)} \n\
    Frase de ejemplo: {ejemplo} \n\
    Frase de ejemplo preprocesada: {preprocessor.apply([ejemplo])} \n'
)


    Largo del diccionario: 24247 
    Frase de ejemplo: Fer estamos para la de 21 hs 
    Frase de ejemplo preprocesada: [] 



### Algoritmo

El algoritmo a implementar es un [clasificador bayesiano naive](https://es.wikipedia.org/wiki/Clasificador_bayesiano_ingenuo). Con la incorporación de un meta-pararametro `N`, que indica la cantidad de palabras anteriores a considerar para la predicción de la siguiente palabra. 

En la letra del laboratorio se menciona que este algoritmo es intensivo en CPU, es parte de la consigna del laboratorio implementar el algoritmo de la manera más eficiente posible.

Detectamos las siguientes tareas para cumplir con la consigna:

- **Definir las estructuras de datos**: Debemos diseñar las estructuras de datos que nos permitan almacenar la información necesaria para el funcionamiento del algoritmo. De forma tal que el acceso a la información y los calculos sean lo más eficientes posibles.
- **Modulo auxiliar `naive_bayes_utils.py`**: Una vez diseñadas las estructuras de datos, debemos implementarlas, proponemos hacerlo siguiendo un enfoque modular, donde se plasmen las ideas discutidas en el punto anterior.
- **Clase `G02NaiveBayesClassifier`**: Una vez implementadas las estructuras de datos, debemos implementar la clase `G02NaiveBayesClassifier` que encapsula el algoritmo de clasificación y mantiene todo el aparato auxiliar encapsulado bajo la misma clase. Esta clase será definida en el modulo `bayesian_learning.py`.


#### Definir las estructuras de datos

Nuestro modelo busca sugerir la palabra más probable dado un contexto de N palabras anteriores. Esto es coincidente con el objetivo mas general de los clasificadores bayesianos, que es encontrar la hipótesis más probable dado un conjunto de evidencias. (`h_map`)

- `h_map` = argmax<sub>h</sub> P(h|D) = argmax<sub>h</sub> P(D|h) P(h) / P(D)

Los clasificadores bayesianos naive asumen que las evidencias son independientes entre si, por lo que la probabilidad de la hipótesis MAP se puede expresar como el producto de las probabilidades de cada evidencia dada la hipótesis multiplicado por la probabilidad de la hipótesis.

- `h_map` = argmax<sub>h</sub> P(D|h) P(h) / P(D) = argmax<sub>h</sub> P(D|h) P(h) = argmax<sub>h</sub> P(d<sub>1</sub>|h) ... P(d<sub>n</sub>|h) P(h)

Nuestro meta-parámetro `N` indica la cantidad de palabras anteriores a considerar para la predicción de la siguiente palabra. Es decir, `N` es la cantidad de evidencias que consideramos para calcular la probabilidad de la hipótesis (la palabra a sugerir).

Si bien un modelo bayesiano naive simplifica los calculos, diseñar estructuras de datos y algoritmos eficientes para calcular las probabilidades no es trivial. Proponemos las siguientes estructuras de datos para almacenar la información necesaria para el funcionamiento del algoritmo:

- `V`: Vocabulario de palabras en el conjunto de entrenamiento.
- `F_h`: Frecuencia de cada hipótesis (palabra) en el conjunto de entrenamiento.
- `F_hD`: Frecuencia de cada evidencia (palabra anterior) dado una hipótesis (palabra) en el conjunto de entrenamiento. A diferencia de `F_h`, `F_hD` es un diccionario de diccionarios y es afectado por el meta-parámetro `N`, ya que la cantidad de evidencias a considerar depende de `N`. Por ejemplo, si `N=4` y la hipótesis es `h`, `F_hD[h]` es un diccionario que contiene la frecuencia de cada combinación de 4 palabras anteriores a `h` en el conjunto de entrenamiento. Esto implica tambien que las frases de longitud menor a `N` no aportan información para el entrenamiento del modelo.

Para calcular las probabilidades usamos el m-estimador como estimador de máxima verosimilitud y suavizador de hipotesis que no aparecen en el conjunto de entrenamiento hasta el momento de la predicción.

#### Modulo auxiliar `naive_bayes_utils.py`

El modulo `naive_bayes_utils.py` encapsula las estructuras de datos y los algoritmos auxiliares necesarios para el funcionamiento del clasificador bayesiano naive. 

Estas estructuras de datos fueron definidas en el punto anterior y fueron implementadas en el modulo `naive_bayes_utils.py` de la siguiente manera:

- `count_words(**args)`: Esta función recibe los datos de entrenamiento preprocesados y el meta-parámetro `N` y devuelve las estructuras de datos `V`, `F_h` y `F_hD` definidas anteriormente.
- `p_h(**args)`: Esta función recibe las estructuras de datos `V`, `F_h` y `F_hD` y `m` y devuelve un estimador de máxima verosimilitud de la probabilidad de cada hipótesis (palabra) en el conjunto de entrenamiento.
- `p_hD(**args)`: Esta función recibe las estructuras de datos `V`, `F_h` y `F_hD` y `m` y devuelve un estimador de máxima verosimilitud de la probabilidad de cada evidencia (palabra anterior) dado una hipótesis (palabra) en el conjunto de entrenamiento.

In [5]:
from src.naive_bayes_utils import *

V, F_h, F_hD = build(preprocessor.apply(data), N=4)

results = {
    'V': V,
    'F_h': F_h,
    'F_hD': F_hD,
}

for key, value in results.items():
  print(f'{key}: {value}')


V: 12634
F_h: Counter({'hoy': 1257, 'horarios': 593, 'disponibles': 585, 'si': 557, 'gracias': 398, 'buenas': 304, 'cancha': 286, 'mensaje': 181, 'elimino': 157, 'libero': 137, 'miercoles': 134, 'falta': 131, 'dia': 119, 'bien': 116, 'hora': 114, 'lunes': 109, 'pronto': 108, 'fernando': 105, 'viernes': 103, 'jorge': 101, 'martes': 101, 'nico': 99, 'jueves': 98, 'buen': 91, 'juan': 86, 'ahi': 83, 'completo': 81, 'mas': 77, 'bueno': 77, 'jugar': 75, 'alejandro': 75, 'sale': 73, 'alguien': 72, 'horas': 68, 'manana': 68, 's': 67, 'alguno': 66, 'reservas': 64, 'arriba': 64, 'vemos': 63, 'partido': 62, 'principiante': 62, 'nando': 57, 'ivan': 56, 'sabado': 53, 'dias': 53, 'muchas': 52, 'gente': 51, 'marco': 51, 'dos': 50, 'libre': 48, 'quedo': 48, 'javier': 47, 'pareja': 46, 'canchas': 46, 'voy': 45, 'lino': 44, 'domingo': 44, 'estan': 44, 'perfecto': 43, 'tambien': 43, 'tardes': 43, 'vamos': 42, 'ver': 42, 'buenos': 39, 'solo': 36, 'ahora': 36, 'sumo': 35, 'amigo': 35, 'rey': 34, 'jajaja': 

#### Clase `G02NaiveBayesClassifier`

La clase `G02NaiveBayesClassifier` es una implementación de un clasificador Naive Bayes. Aquí hay un resumen de sus componentes principales:

- `__init__`: El constructor toma un conjunto de datos(`data`) y un parámetro `N` que indica cuántas palabras anteriores se deben considerar para la predicción. El constructor también inicializa el preprocesador y construye las estructuras de datos necesarias (`V`, `F_h`, `F_hD`) utilizando la función `build`.

- `predict`: Esta función toma una oración y devuelve la palabra más probable según el modelo Naive Bayes. Utiliza el logaritmo de las probabilidades para evitar problemas de precisión numérica.

- `update`: Esta función actualiza el modelo con una nueva oración. Actualiza las frecuencias de las palabras y las frecuencias condicionales, así como el tamaño total del vocabulario (`V`).

La clase utiliza funciones auxiliares como `p_h` y `p_hD`, definidas en el módulo `naive_bayes_utils.py`, para calcular las probabilidades de las palabras.

```python
def predict(self, sentence):
    argmax_h = {}

    for h in self.F_h.keys():
        p = log(p_h(h, self.V, self.F_h))

        for word in sentence:
            p += log(p_hD(word, h, self.V, self.F_h, self.F_hD))

        argmax_h[h] = p
    
    return max(argmax_h, key=argmax_h.get)
    
def update(self, new_sentence):
    self.F_h.update(new_sentence)

    for i in range(self.N, len(new_sentence)):
        current_word = new_sentence[i]
        previous_words = tuple(new_sentence[i-self.N:i])

        for previous_word in previous_words:
            self.F_hD[current_word].update([previous_word])
    
    self.V = sum(self.F_h.values())
```

In [ ]:
from src.bayesian_learning import G02NaiveBayesClassifier
import time

start = time.time()
clf = G02NaiveBayesClassifier(data, N=2)
print(f'[INIT]  Tiempo de ejecución: {time.time() - start:.4f}s')
test = ['ejemplo']

print(' '.join(test))

start = time.time()
for i in range(2): test.append(clf.predict(test))
print(f'[PREDICT] Tiempo de ejecución: {time.time() - start:.4f}s')

print(f'        V: {clf.V} \
    \n          F_h[{test[1]}]: {clf.F_h[test[1]]} \
    \n          F_hD[{test[1]}][{test[0]}]: {clf.F_hD[test[1]][test[0]]} \
    \n          F_h[{test[2]}]: {clf.F_h[test[2]]} \
    \n          F_hD[{test[2]}][{test[1]}]: {clf.F_hD[test[2]][test[1]]}')

start = time.time()

print(' '.join(test))

clf.update(test)
print(f'[UPDATE]  Tiempo de ejecución: {time.time() - start:.4f}s')

print(f'        V: {clf.V} \
    \n          F_h[{test[1]}]: {clf.F_h[test[1]]} \
    \n          F_hD[{test[1]}][{test[0]}]: {clf.F_hD[test[1]][test[0]]} \
    \n          F_h[{test[2]}]: {clf.F_h[test[2]]} \
    \n          F_hD[{test[2]}][{test[1]}]: {clf.F_hD[test[2]][test[1]]}')

### Evaluación

En cualquier laboratorio o proyecto de investigación, la sección de evaluación es crucial para entender cómo se mide el rendimiento del modelo y qué métricas son relevantes para el problema en cuestión. Sin embargo, la naturaleza planteada en la letra del laboratorio no permite una evaluación cuantitativa del modelo. Por lo tanto, proponemos una evaluación cualitativa del modelo.

No sabemos como cuantificar el hecho de que el modelo esta "sugeriendo bien" dado que es una percepción subjetiva del usuario que esta manteniendo una conversación con el modelo.

Lo que si podemos hacer, adoptando una evaluacion ya mas relajada. Dada una sesion de cliente, cuantificar la cantidad de veces que el modelo sugiere una palabra que el usuario considera correcta.

Alterando el tamaño de la ventana de palabras `N`, y el suavizado `M` buscamos encontrar un equilibrio entre las siguientes propiedades deseables:

- **Memoria**: El modelo debe ser capaz de sugerir palabras que tengan coherencia con el grupo de Whatsapp de donde se extrajeron los datos.

- **Adaptacion**: El modelo debe ser capaz de adaptarse a la nueva evidencia, y mejorar su performance, sugiriendo palabras que tengan coherencia con la sesion de chat que se esta simulando.

Esta particularidad en la evaluación del modelo, tambien impacta en la definición de los conjuntos de entrenamiento, prueba y evaluación.

El clasificador bayesiano se construye a partir de un conjunto de entrenamiento. En nuestro caso, el conjunto de entrenamiento es el conjunto de mensajes de un grupo de WhatsApp. No hay conjunto de prueba ni conjunto de evaluación. El modelo se evalúa en tiempo real, mientras el usuario (en primera instancia nosotros y luego el cuerpo docente) interactúa con el modelo y evalúa la calidad de las sugerencias.

## Experimentación

### Simulador de cliente.

El experimento consiste en evaluar los distintos hiperparametros del modelo.

El siguiente script permite simular el comportamiento de un cliente que escribe frases, fue proporcionado por el cuerpo docente. Es de utilidad para experimentar con modelo, ya que permite ingresar frases y ver las sugerencias que el modelo realiza.

**Nota:**

El comportamiento del siguiente script en `VSCode` es diferente al comportamiento en la terminal. En VSCode, el script omite entradas, se desfasa y hace que el analisis de los resultados sea dificil. Por lo tanto, recomendamos ejecutar el script en la terminal. (`client.py`)

## Conclusión

Una breve conclusión del trabajo realizado. Por ejemplo: 
- ¿cuándo se dieron los mejores resultados del jugador?
- ¿encuentra alguna relación con los parámetros / oponentes/ atributos elegidos?
- ¿cómo mejoraría los resultados?